In [1]:
from isaacgym import gymtorch
from isaacgym import gymapi

from isaacgymenvs.utils import reformat
from isaacgymenvs.utils.rlgames_utils import RLGPUAlgoObserver
from isaacgymenvs.utils.rlgames_utils import get_rlgames_env_creator
from isaacgymenvs import train
import isaacgymenvs
from rl_games.torch_runner import Runner

from omegaconf import OmegaConf
from hydra import compose, initialize
import yaml
import torch
import numpy as np
import time
import os
os.environ['DISPLAY'] = ":1"

Importing module 'gym_37' (/scr-ssd/ksrini/Downloads/isaacgym/python/isaacgym/_bindings/linux-x86_64/gym_37.so)
Setting GYM_USD_PLUG_INFO_PATH to /scr-ssd/ksrini/Downloads/isaacgym/python/isaacgym/_bindings/linux-x86_64/usd/plugInfo.json
PyTorch version 1.8.1
Device count 1
/scr-ssd/ksrini/Downloads/isaacgym/python/isaacgym/_bindings/src/gymtorch
Using /afs/cs.stanford.edu/u/ksrini/.cache/torch_extensions as PyTorch extensions root...
Emitting ninja build file /afs/cs.stanford.edu/u/ksrini/.cache/torch_extensions/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module gymtorch...


/scr-ssd/ksrini/Downloads/isaacgym/python/isaacgym/torch_utils.py:135: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def get_axis_params(value, axis_idx, x_value=0., dtype=np.float, n_dims=3):
2022-07-02 19:08:51,981 - INFO - logger - logger initialized


Error: FBX library failed to load - importing FBX data will not succeed. Message: No module named 'fbx'
FBX tools must be installed from https://help.autodesk.com/view/FBX/2020/ENU/?guid=FBX_Developer_Help_scripting_with_python_fbx_installing_python_fbx_html


<unknown>:6: DeprecationWarning: invalid escape sequence \*


In [2]:
with initialize(config_path="cfg", job_name="test_env"):
    cfg = compose(config_name="config", overrides=["task=AllegroHandGrasp", "task.env.observationType=full",
                                                   "sim_device=cpu", "test=true", "checkpoint='./runs/juno-runs/AllegroHand/nn/AllegroHand.pth'",
                                                   "num_envs=2"])

env = isaacgymenvs.make(cfg.seed, cfg.task, cfg.num_envs, cfg.sim_device,
                  cfg.rl_device, cfg.graphics_device_id, cfg.headless,
                  cfg.multi_gpu, cfg.capture_video, cfg.force_render, cfg)

/scr1/.pyenv/versions/miniconda3-latest/envs/rlgpu/lib/python3.7/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
/scr1/.pyenv/versions/miniconda3-latest/envs/rlgpu/lib/python3.7/site-packages/hydra/_internal/defaults_list.py:413: UserWarning: In config: Invalid overriding of hydra/job_logging:
Default list overrides requires 'override' keyword.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/defaults_list_override for more information.

  deprecation_warning(msg)
/scr1/.pyenv/versions/miniconda3-latest/envs/rlgpu/lib/python3.7/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


Obs type: full
GPU Pipeline can only be used with GPU simulation. Forcing CPU Pipeline.
Not connected to PVD
Physics Engine: PhysX
Physics Device: cpu
GPU Pipeline: disabled
Num dofs:  16
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Num dofs:  16


env.shadow_hand_dof_pos[0].detach().cpu() - old

old = env.shadow_hand_dof_pos[0].detach().cpu().numpy()

In [3]:
def get_action(t):
    hand_pos = np.zeros((env.num_envs, 3))
    hand_pos[:, 2] = np.sin(t/120 * np.pi) * 0.25 + 0.25
    actions = np.stack([env.action_space.sample() for _ in range(env.num_envs)])
    # actions[:, :3] = hand_pos
    return torch.tensor(actions, device=env.device).float()

## Get camera transform

Saving camera transform

```python
transform = env.gym.get_viewer_camera_transform(env.viewer, env.envs[0])
pos = transform.p
print("position", pos.x, pos.y, pos.z)

rot = transform.r
print("rotation", rot.x, rot.y, rot.z, rot.w)

import pickle
pickle.dump(transform, open('allegro_viewer.pkl', 'wb'))
```

```python
import pickle

transform = pickle.load(open('allegro_viewer.pkl', 'rb'))
cam_pos = transform.p
cam_target = gymapi.Vec3()
env.gym.viewer_camera_look_at(env.viewer, None, cam_pos, cam_target)
```

In [4]:
obs = env.reset()
action = get_action(0)
obs, r, done, info = env.step(action)

t = 0

# moves hand up and down
while True:
    t += 1
    action = get_action(0)
    obs, r, done, info = env.step(action)
    env_ids = done.nonzero(as_tuple=False).squeeze(-1)
    goal_env_ids = env.reset_goal_buf.nonzero(as_tuple=False).squeeze(-1)
    # time.sleep(0.01)
    if done.any():
        env.reset_idx(env_ids, goal_env_ids)

SystemExit: 

/scr1/.pyenv/versions/miniconda3-latest/envs/rlgpu/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
